# Project: Global fertility rate data analysis

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

## Introduction <a id='intro'></a>

This project aims at exploring global fertility data that was collected from the [Gapminder project](www.gapminder.org). Apart from the fertility data itself, I downloaded four other datasaets with indicators that I assume might be connected to the fertility: 

* child mortality (0-5 year olds dying per 1000 born)
* income per person (GDP/capita, PPP\$ inflation-adjusted)
* women's education (average years spent in school by women aged between 15 and 44 years)
* urban population (\% of total population living in urban areas)

In order to categorise the countries contained in the data as regions, I also downloaded [geographical data](https://www.gapminder.org/data/geo/) including these categories.

The idea is to explore how the fertility rate changed over the past 220 years in the different regions of the world. Furthermore I would like to find out, if the above indicators are correlated with the fertility rate and if the correlation is different depending on the country and/or region investigated. 

In [26]:
# import packages
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

<a id='wrangling'></a>
## Data Wrangling

### General Properties

In [27]:
# load fertility data
df_fertility = pd.read_csv('fertility.csv')
df_fertility.head()

,country,1800,1801,1802,1803,1804,1805,1806,1807,1808,...,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100
0,Afghanistan,7.00,7.00,7.00,7.00,7.00,7.00,7.00,7.00,7.00,...,1.74,1.74,1.74,1.74,1.74,1.74,1.74,1.74,1.74,1.74
1,Albania,4.60,4.60,4.60,4.60,4.60,4.60,4.60,4.60,4.60,...,1.78,1.78,1.78,1.79,1.79,1.79,1.79,1.79,1.79,1.79
2,Algeria,6.99,6.99,6.99,6.99,6.99,6.99,6.99,6.99,6.99,...,1.86,1.86,1.86,1.86,1.86,1.86,1.86,1.86,1.86,1.86
3,Angola,6.93,6.93,6.93,6.93,6.93,6.93,6.93,6.94,6.94,...,2.54,2.52,2.50,2.48,2.47,2.45,2.43,2.42,2.40,2.40
4,Antigua and Barbuda,5.00,5.00,4.99,4.99,4.99,4.98,4.98,4.97,4.97,...,1.81,1.81,1.81,1.81,1.81,1.81,1.81,1.82,1.82,1.82


In [28]:
# check data types
df_fertility.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184 entries, 0 to 183
Columns: 302 entries, country to 2100
dtypes: float64(301), object(1)
memory usage: 434.2+ KB


The above output shows that the countries are stored as strings and the fertility rates are stored as floats for all countries. No adjustments needed. 
***

In [29]:
# check for missing values
sum(df_fertility.isnull().any())

0

The above output shows that there is no missing values in the data.
***

In [30]:
# load child mortality data
df_mortality = pd.read_csv('child_mortality.csv')
df_mortality.head()

,country,1800,1801,1802,1803,1804,1805,1806,1807,1808,...,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100
0,Afghanistan,469.0,469.0,469.0,469.0,469.0,469.0,470.0,470.0,470.0,...,12.60,12.40,12.20,12.00,11.80,11.60,11.50,11.30,11.10,11.10
1,Albania,375.0,375.0,375.0,375.0,375.0,375.0,375.0,375.0,375.0,...,2.32,2.30,2.27,2.24,2.22,2.19,2.16,2.14,2.11,2.11
2,Algeria,460.0,460.0,460.0,460.0,460.0,460.0,460.0,460.0,460.0,...,4.28,4.20,4.14,4.07,4.00,3.94,3.88,3.82,3.76,3.76
3,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.86,0.84,0.83,0.81,0.80,0.79,0.78,0.77,0.76,0.76
4,Angola,486.0,486.0,486.0,486.0,486.0,486.0,486.0,486.0,486.0,...,17.70,17.50,17.30,17.10,17.00,16.80,16.60,16.40,16.30,16.30


In [31]:
# check data types
df_mortality.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Columns: 302 entries, country to 2100
dtypes: float64(301), object(1)
memory usage: 460.2+ KB


The above output shows that the countries are stored as strings and the mortality rates are stored as floats for all countries. No adjustments needed here. 
***

In [32]:
# check for missing values
sum(df_mortality.isnull().any())

150

This output shows that there are 150 missing values in the data. Since I can see above that there are NaNs for the country Andorra, I am wondering if all the 150 missing values are in that row. 
***

In [33]:
# check missing values for the first 150 data columns of the Andorra row
sum(df_mortality.iloc[3,1:151].isnull())

150

This shows that Andorra is the only country with missing values. As there is no fertility data for Andorra, I will not need this row for my analysis.
***

In [34]:
# load income data
df_income = pd.read_csv('income_per_person.csv')
df_income.head()

,country,1800,1801,1802,1803,1804,1805,1806,1807,1808,...,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040
0,Afghanistan,603,603,603,603,603,603,603,603,603,...,2550,2600,2660,2710,2770,2820,2880,2940,3000,3060
1,Albania,667,667,667,667,667,668,668,668,668,...,19400,19800,20200,20600,21000,21500,21900,22300,22800,23300
2,Algeria,715,716,717,718,719,720,721,722,723,...,14300,14600,14900,15200,15500,15800,16100,16500,16800,17100
3,Andorra,1200,1200,1200,1200,1210,1210,1210,1210,1220,...,73600,75100,76700,78300,79800,81500,83100,84800,86500,88300
4,Angola,618,620,623,626,628,631,634,637,640,...,6110,6230,6350,6480,6610,6740,6880,7020,7160,7310


In [35]:
# check data types
df_income.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Columns: 242 entries, country to 2040
dtypes: int64(241), object(1)
memory usage: 365.0+ KB


The above output shows that the countries are stored as strings and the income data is stored as ints for all countries. No adjustments needed here. 
***

In [36]:
# check for missing values
sum(df_income.isnull().any())

0

The above output shows that there is no missing values in the data.
***

In [37]:
# load education data
df_edu = pd.read_csv('education.csv')
df_edu.head()

,country,1970,1971,1972,1973,1974,1975,1976,1977,1978,...,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009
0,Afghanistan,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.2,...,0.5,0.5,0.5,0.6,0.6,0.6,0.7,0.7,0.7,0.8
1,Albania,5.6,5.7,5.9,6.0,6.2,6.3,6.5,6.6,6.8,...,9.8,9.9,10.0,10.1,10.2,10.3,10.4,10.5,10.6,10.7
2,Algeria,1.4,1.5,1.6,1.7,1.8,1.9,2.1,2.2,2.3,...,5.8,5.9,6.1,6.2,6.4,6.5,6.7,6.8,6.9,7.1
3,Angola,0.9,1.0,1.0,1.1,1.1,1.2,1.2,1.3,1.4,...,3.5,3.6,3.7,3.8,3.9,4.0,4.1,4.3,4.4,4.5
4,Antigua and Barbuda,8.1,8.3,8.5,8.7,8.8,9.0,9.2,9.4,9.6,...,12.7,12.8,12.9,13.0,13.1,13.2,13.3,13.3,13.4,13.5


In [38]:
# check data types
df_edu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174 entries, 0 to 173
Data columns (total 41 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   country  174 non-null    object 
 1   1970     174 non-null    float64
 2   1971     174 non-null    float64
 3   1972     174 non-null    float64
 4   1973     174 non-null    float64
 5   1974     174 non-null    float64
 6   1975     174 non-null    float64
 7   1976     174 non-null    float64
 8   1977     174 non-null    float64
 9   1978     174 non-null    float64
 10  1979     174 non-null    float64
 11  1980     174 non-null    float64
 12  1981     174 non-null    float64
 13  1982     174 non-null    float64
 14  1983     174 non-null    float64
 15  1984     174 non-null    float64
 16  1985     174 non-null    float64
 17  1986     174 non-null    float64
 18  1987     174 non-null    float64
 19  1988     174 non-null    float64
 20  1989     174 non-null    float64
 21  1990     174 non

The above output shows that the countries are stored as strings and the education data is stored as floats for all countries. Furthermore it turns out that there is only data for 1970-2009. Concerning education, my analysis will be limited to these years. 

I can see in the ouput that there are no missing values.
***

In [39]:
# load urban population data
df_urban = pd.read_csv('urban_population.csv')
df_urban.head()

,country,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Afghanistan,0.084,0.0868,0.0898,0.0928,0.0959,0.099,0.102,0.106,0.109,...,0.237,0.239,0.242,0.244,0.246,0.248,0.250,0.253,0.255,0.258
1,Albania,0.307,0.3090,0.3100,0.3110,0.3120,0.312,0.313,0.314,0.314,...,0.522,0.532,0.543,0.554,0.564,0.574,0.584,0.594,0.603,0.612
2,Algeria,0.305,0.3180,0.3320,0.3470,0.3610,0.376,0.388,0.390,0.392,...,0.675,0.682,0.689,0.696,0.702,0.708,0.715,0.721,0.726,0.732
3,Andorra,0.585,0.6100,0.6350,0.6590,0.6820,0.704,0.726,0.746,0.766,...,0.888,0.887,0.886,0.885,0.884,0.883,0.882,0.882,0.881,0.880
4,Angola,0.104,0.1080,0.1120,0.1160,0.1210,0.125,0.130,0.134,0.139,...,0.598,0.605,0.613,0.620,0.627,0.634,0.641,0.648,0.655,0.662


In [40]:
# check data types
df_urban.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194 entries, 0 to 193
Data columns (total 61 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   country  194 non-null    object 
 1   1960     192 non-null    float64
 2   1961     192 non-null    float64
 3   1962     192 non-null    float64
 4   1963     192 non-null    float64
 5   1964     192 non-null    float64
 6   1965     192 non-null    float64
 7   1966     192 non-null    float64
 8   1967     192 non-null    float64
 9   1968     192 non-null    float64
 10  1969     192 non-null    float64
 11  1970     192 non-null    float64
 12  1971     192 non-null    float64
 13  1972     192 non-null    float64
 14  1973     192 non-null    float64
 15  1974     192 non-null    float64
 16  1975     192 non-null    float64
 17  1976     192 non-null    float64
 18  1977     192 non-null    float64
 19  1978     192 non-null    float64
 20  1979     192 non-null    float64
 21  1980     192 non

The above output shows that the countries are stored as strings and the urbanity data is stored as floats for all countries. Furthermore it turns out that there is only data for 1960-2019. Concerning urbanity, my analysis will be limited to these years.

I can see that there is data for 194 countries between 1990 and 2011 but only for 192 and 193 countries before and after that time period. Since there is fertility data for only 184 countries this will not be a problem for the analysis.
***

In [44]:
# load geographical data
df_geo = pd.read_csv('country_regions.csv',sep=';')
df_geo.head()

,geo,name,four_regions,eight_regions,six_regions,members_oecd_g77,Latitude,Longitude,UN member since,World bank region,"World bank, 4 income groups 2017","World bank, 3 income groups 2017"
0,afg,Afghanistan,asia,asia_west,south_asia,g77,33,66,19.11.1946,South Asia,Low income,NaN
1,alb,Albania,europe,europe_east,europe_central_asia,others,41,20,14.12.1955,Europe & Central Asia,Upper middle income,NaN
2,dza,Algeria,africa,africa_north,middle_east_north_africa,g77,28,3,8.10.1962,Middle East & North Africa,Upper middle income,NaN
3,and,Andorra,europe,europe_west,europe_central_asia,others,"42,50779","1,52109",28.7.1993,Europe & Central Asia,High income,NaN
4,ago,Angola,africa,africa_sub_saharan,sub_saharan_africa,g77,"-12,5","18,5",1.12.1976,Sub-Saharan Africa,Lower middle income,NaN


In [49]:
# check data types in the columns that will be used
df_geo[['name','eight_regions']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197 entries, 0 to 196
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   name           197 non-null    object
 1   eight_regions  197 non-null    object
dtypes: object(2)
memory usage: 3.2+ KB


This output shows that both the country names and the region categories are stored as strings and that there are no missing values. No adjustments needed.

### Data Cleaning

In [ ]:
# After discussing the structure of the data and any problems that need to be
#   cleaned, perform those cleaning steps in the second part of this section.


<a id='eda'></a>
## Exploratory Data Analysis

> **Tip**: Now that you've trimmed and cleaned your data, you're ready to move on to exploration. Compute statistics and create visualizations with the goal of addressing the research questions that you posed in the Introduction section. It is recommended that you be systematic with your approach. Look at one variable at a time, and then follow it up by looking at relationships between variables.

### Research Question 1 (Replace this header name!)

In [ ]:
# Use this, and more code cells, to explore your data. Don't forget to add
#   Markdown cells to document your observations and findings.


### Research Question 2  (Replace this header name!)

In [ ]:
# Continue to explore the data to address your additional research
#   questions. Add more headers as needed if you have more questions to
#   investigate.


<a id='conclusions'></a>
## Conclusions

> **Tip**: Finally, summarize your findings and the results that have been performed. Make sure that you are clear with regards to the limitations of your exploration. If you haven't done any statistical tests, do not imply any statistical conclusions. And make sure you avoid implying causation from correlation!

> **Tip**: Once you are satisfied with your work, you should save a copy of the report in HTML or PDF form via the **File** > **Download as** submenu. Before exporting your report, check over it to make sure that the flow of the report is complete. You should probably remove all of the "Tip" quotes like this one so that the presentation is as tidy as possible. Congratulations!